In [ ]:
# !pip install boto3
# !pip install python-dotenv

In [ ]:
import run_services as rn

proc = rn.start_services()

In [ ]:
rn.stop_services(proc)

In [1]:
import importlib

# flights
import flights.aggregator as ag
from datetime import date, timedelta

# video
import media.video_generator as vg

# caption
import content.caption_builder as cb

# instagram
from pathlib import Path
import os
from instagram.ig_client import InstagramClient

# review
import uuid
from pathlib import Path
import review.telegram_review as tr

import affiliates.affiliates as af


## FLIGHTS

In [2]:
start = date.today()
end = start + timedelta(days=7)

flights = ag.get_flights_in_period(start, end)
best_by_cat = ag.get_best_by_category_scored(flights)
# best_by_cat = ag.get_best_by_category_scored(flights[25:35])



for item in best_by_cat:
    f = item["flight"]
    cat = item["category"]
    score = item["score"]
    print(cat["label"], f"{f.origin}→{f.destination}", f.price, f.price_per_km, score)

🔎 Buscando vuelos entre 2025-12-02 y 2025-12-09...
🗓  Buscando vuelos 2025-12-04 → 2025-12-07...
🗓  Buscando vuelos 2025-12-04 → 2025-12-08...
🗓  Buscando vuelos 2025-12-05 → 2025-12-07...
🗓  Buscando vuelos 2025-12-05 → 2025-12-08...
✅ Encontrados 94 vuelos en total.
✨ Escapada Perfecta PMI→FMO 173.03 0.1210087150952134 4.134823472340229
🍝 Escapada Gastronómica PMI→WAW 184.63 0.09301612099190701 5.3781204094993695
🏛 Escapada Cultural PMI→VIE 244.3 0.16756112861652123 2.986032294306118
❤️ Escapada Romántica PMI→VIE 194.41000000000003 0.13334244377543142 3.7523156704849776
🎉 Finde Perfecto PMI→VIE 262.18 0.17982471019516796 2.7823925909641645


In [ ]:
# best_by_cat

In [3]:
main_item = ag.choose_main_candidate_prob(best_by_cat)

# main_item = best_by_cat[1]

if main_item is None:
    raise ValueError("No hay candidatos para hoy")

main_flight = main_item["flight"]
main_category_code = main_item["category"]["code"]

main_flight

Flight(origin='PMI', destination='WAW', price=184.63, start_date='2025-12-05 05:45:00', end_date='2025-12-07 16:00:00', airline='Ryanair', link='https://www.ryanair.com/es/es/trip/flights/select?adults=1&teens=0&children=0&infants=0&dateOut=2025-12-05&dateIn=2025-12-07&isConnectedFlight=false&discount=0&promoCode=&isReturn=true&originIata=PMI&destinationIata=WAW&tpAdults=1&tpTeens=0&tpChildren=0&tpInfants=0&tpStartDate=2025-12-05&tpEndDate=2025-12-07&tpDiscount=0&tpPromoCode=&tpOriginIata=PMI&tpDestinationIata=WAW', distance_km=1984.9247424117373, price_per_km=0.09301612099190701)

In [4]:
NGROK_BASE_URL = "https://karie-apocatastatic-maddox.ngrok-free.dev" 
BASE_DIR = "/media/videos/reel.mp4"

video_url = NGROK_BASE_URL + BASE_DIR

In [5]:
from web.exporter import update_flights_json
# import flights.api_kiwi as ak
import affiliates.affiliates as af

# SKYSCANNER_MEDIA_PARTNER_ID = "AFF_TRA_19354_00001"

importlib.reload(af)



from affiliates.affiliates import build_affiliate_url_for_flight

flight = main_item["flight"]   # tu Flight(...)
affiliate_url = build_affiliate_url_for_flight(flight)

# update_flights_json(
#     main_item=main_item,
#     json_path=Path("web/mallorca/flights_of_the_day.json"),
#     market="PMI",
#     reel_url=video_url,
#     affiliate_url=affiliate_url,
#     max_entries=5,
# )

import web.exporter as ex
import web.uploader as up

importlib.reload(ex)
importlib.reload(up)



data = ex.update_flights_json(
    main_item=main_item,
    json_path="local_copy.json",  # opcional
    market="PMI",
    reel_url=video_url,
    affiliate_url=affiliate_url,
    max_entries=5,
)

up.upload_flights_json(data, key="pmi/flights_of_the_day.json")


JSON subido a: https://escapadasgo-public.s3.eu-west-1.amazonaws.com/pmi/flights_of_the_day.json


In [ ]:
affiliate_url

In [ ]:

# 7) Actualizar JSON de la web
update_flights_json(
    main_item=main_item,
    json_path=Path("web/mallorca/flights_of_the_day.json"),
    market="PMI",
    reel_url=video_url,
    affiliate_url=affiliate_url,         # aquí puedes meter tu link de Skyscanner/Tequila
    max_entries=5,
)

## VIDEO

In [ ]:
importlib.reload(vg)

In [ ]:
logo_path = "media/images/EscapGo_circ_logo_transparent.png"   # circular, transparente
out_path = "media/videos/reel.mp4"

vg.create_reel_for_flight(
    main_flight,               # tu objeto Flight o dict
    out_mp4_path=out_path,
    logo_path=logo_path,
    duration=4.0,
)


## CAPTION

In [ ]:
caption_text = cb.build_caption_for_flight(
    main_flight,
    category_code=main_category_code,  # "cultural", "romantica", etc.
    tone="emocional",
)

caption_text

## REVIEW

In [ ]:
# Después de elegir tu "mejor vuelo", generar caption y vídeo:
job_id = str(uuid.uuid4())

local_video_path = Path("media/videos/reel.mp4")

review_candidates = tr.to_review_candidates(best_by_cat)

tr.register_job(
    job_id=job_id,
    flight=best_by_cat[0]['flight'],
    caption=caption_text,
    video_path=local_video_path,
    candidates=review_candidates,  # si quieres pasarle más opciones
)

tr.send_review_candidate(job_id)
print(f"Enviado a Telegram para revisión. job_id={job_id}")

## INSTAGRAM

In [ ]:
caption_text = "Test Reel desde @escapadas_mallorca 🚀"

# 4) Cliente IG
ig = InstagramClient()

# 5) Crear contenedor
creation_id = ig.create_reel_container(video_url=video_url, caption=caption_text)

# 6) Esperar a que esté listo
if ig.wait_until_ready(creation_id):
    # 7) Publicar
    # reel_id = ig.publish_reel(creation_id)
    # permalink = ig.get_media_permalink(reel_id)
    print("✅ Reel publicado:")
    # print("ID:", reel_id)
    # print("Permalink:", permalink)
else:
    print("❌ No se pudo procesar el vídeo, no se publica.")

In [ ]:
import pandas as pd
pd.DataFrame(flights).sort_values(by='price')

In [ ]:
# main.py

import time
from datetime import date

from config.settings import SETTINGS
from flights.aggregator import get_best_daily_flight
from content.caption_builder import build_caption
from media.image_picker import pick_image_for_destination
from media.video_generator import generate_reel_video
from storage.uploader import upload_video_and_get_url
from instagram.publish import publish_reel
from review.telegram_bot import send_review_message, wait_for_user_decision

In [ ]:
def daily_workflow():
    print("🚀 Iniciando flujo diario...")

    # 1) Obtener mejor oferta del día (de cualquier API disponible)
    print("🔍 Buscando mejor vuelo del día...")
    flight = get_best_daily_flight(date.today())
    if not flight:
        print("❌ No se encontró ningún vuelo.")
        return

    # 2) Generar caption con OpenAI (JSON estrictamente controlado)
    print("✍️ Generando caption...")
    caption_json = build_caption(flight)
    caption_text = caption_json_to_string(caption_json)  # si necesitas convertir

    # 3) Seleccionar imagen asociada al destino
    print("🖼 Seleccionando imagen...")
    image_path = pick_image_for_destination(flight.destination)

    # 4) Generar vídeo estilo Reel (6s, zoom, texto)
    print("🎬 Generando vídeo...")
    video_local_path = generate_reel_video(image_path, flight)

    # 5) Enviar todo a Telegram para revisión manual
    print("📲 Enviando a Telegram para revisión...")
    send_review_message(flight, caption_text, video_local_path)

    # 6) Esperar la decisión humana (bloqueante o async)
    print("⌛ Esperando decisión del usuario...")
    decision = wait_for_user_decision()

    if decision == "approve":
        print("👍 Usuario aprobó. Publicando en Instagram...")

        # 7) Subir vídeo a una URL pública
        print("🌐 Subiendo vídeo...")
        video_url = upload_video_and_get_url(video_local_path)

        # 8) Publicar en Instagram
        print("📤 Publicando Reel...")
        permalink = publish_reel(video_url, caption_text)

        print("🎉 Publicado con éxito:", permalink)
        return permalink

    elif decision == "reject":
        print("❌ Usuario rechazó el vuelo. Buscando otra oferta...")
        # Recursivo o bucle para repetir
        return daily_workflow()

    else:
        print("⚠️ Decisión desconocida:", decision)


def caption_json_to_string(caption_json: dict) -> str:
    """
    Convierte el JSON de OpenAI al caption final formateado.
    """
    return (
        f"{caption_json['hook']}\n\n"
        f"{caption_json['summary_line']}\n\n"
        f"{caption_json['dates_block']}\n\n"
        f"{caption_json['itinerary_block']}\n\n"
        f"{caption_json['tips_block']}\n\n"
        f"{caption_json['cta_block']}\n\n"
        f"{caption_json['hashtags']}"
    )

In [ ]:

if __name__ == "__main__":
    daily_workflow()

In [ ]:
# import subprocess
# import sys
# import os
# import time
# import signal

# BASE_DIR = r"C:\Users\Macia\Desktop\Jupyter Notebooks\EscapadasMallorca"

# def start_services():
#     cmd = [sys.executable, "run_services.py"]

#     # Lanzamos el proceso, pero sin bloquear el hilo principal
#     proc = subprocess.Popen(
#         cmd,
#         cwd=BASE_DIR,
#         stdout=subprocess.PIPE,
#         stderr=subprocess.PIPE,
#         text=True
#     )
#     print(f"Servicios iniciados (PID {proc.pid})")
#     return proc

# def stop_services(proc):
#     print(f"Deteniendo servicios (PID {proc.pid})...")

#     # En Windows, Popen.terminate() envía una señal de terminación compatible
#     proc.terminate()

#     try:
#         proc.wait(timeout=10)
#         print("Servicios detenidos correctamente.")
#     except subprocess.TimeoutExpired:
#         print("Forzando cierre...")
#         proc.kill()

# if __name__ == "__main__":
#     proc = start_services()

#     print("\nPresiona ENTER para detener todo.\n")
#     input()

#     stop_services(proc)